In [2]:
import openai
import dotenv
import os

if not dotenv.load_dotenv("../../.env"):
    raise ValueError("EnvironmentError: Failed to load `.env`")

api_key = os.getenv("OPENAI_API_KEY") or ""

if not api_key:
    raise ValueError("EnvironmentError: Failed to load `OPENAI_API_KEY`")

openai.api_key = api_key

In [3]:
system_prompt = {
    "role": "system", "content": "My name is ChatGPT. I am a helpful assistant."
}

user_input = {
    "role": "user", "content": "Hello! What is your name?"
}

messages = [system_prompt, user_input]

get_chat_completion = openai.ChatCompletion.create

response = get_chat_completion(model="gpt-3.5-turbo", messages=messages, temperature=0)

messages.append(response["choices"][0]["message"])

print(response)

{
  "id": "chatcmpl-85JcvH0L79CvQBZgkZGASFSDim6no",
  "object": "chat.completion",
  "created": 1696277777,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! My name is ChatGPT. How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 31,
    "completion_tokens": 16,
    "total_tokens": 47
  }
}


In [4]:
for message in messages:
    print(f"{message['role']}\n{message['content']}\n")

system
My name is ChatGPT. I am a helpful assistant.

user
Hello! What is your name?

assistant
Hello! My name is ChatGPT. How can I assist you today?



In [5]:
functions = [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": { "type": "string", "enum": ["celsius", "fahrenheit"] }
        },
        "required": ["location"]
      }
    }
]


def get_current_weather(location: str, unit: str = "celsius"):
    """
    Get the current weather in a given location.

    Parameters:
    location (str): The city and state, e.g. San Francisco, CA
    unit (str): The unit of temperature, can be either 'celsius' or 'fahrenheit'. Default is 'celsius'.

    Returns:
    str: A string that describes the current weather.
    """

    # This is a mock function, so let's return a mock weather report.
    weather_report = f"The current weather in {location} is 20 degrees {unit}."
    return weather_report


In [6]:
user_input = {
    "role": "user", "content": "What is the weather like today in New York City, New York?"
}

messages.append(user_input)

response = get_chat_completion(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)

messages.append(response["choices"][0]["message"])

print(response)

{
  "id": "chatcmpl-85Jd5WnjQwSu8vpAOQTEAe3z2oTg5",
  "object": "chat.completion",
  "created": 1696277787,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"New York City, NY\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 132,
    "completion_tokens": 20,
    "total_tokens": 152
  }
}


In [7]:
for message in messages:
    print(f"{message['role']}\n{message['content']}\n")

system
My name is ChatGPT. I am a helpful assistant.

user
Hello! What is your name?

assistant
Hello! My name is ChatGPT. How can I assist you today?

user
What is the weather like today in New York City, New York?

assistant
None



In [16]:
print(response)

{
  "id": "chatcmpl-85JdQRtxpxGq9Fr9fUbU1xZUdE1BJ",
  "object": "chat.completion",
  "created": 1696277808,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "For a temperature of 20 degrees Celsius in New York City, NY, it is recommended to dress in light clothing such as a t-shirt or blouse with shorts, a skirt, or lightweight pants. Additionally, you may want to bring a light sweater or jacket in case the temperature drops in the evening. It's always a good idea to check the weather forecast throughout the day for any unexpected changes."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 192,
    "completion_tokens": 81,
    "total_tokens": 273
  }
}


In [8]:
function_message = response["choices"][0]["message"]

print(function_message)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"New York City, NY\"\n}"
  }
}


In [9]:
function_call = response["choices"][0]["message"]["function_call"]

print(function_call)

{
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"New York City, NY\"\n}"
}


In [10]:
import json

function_name = function_call["name"]
function_args = json.loads(function_call["arguments"])

print(function_name, function_args)

get_current_weather {'location': 'New York City, NY'}


In [11]:
if function_name == 'get_current_weather':
    location = function_args['location']
    unit = function_args.get('unit', 'celsius')  # use default 'celsius' if unit is not provided
    weather_report = get_current_weather(location, unit)
    # then pass weather_report back to the model as a new message
    print(weather_report)

The current weather in New York City, NY is 20 degrees celsius.


In [12]:
messages[-1]["content"] = weather_report

for message in messages:
    print(f"{message['role']}\n{message['content']}\n")

system
My name is ChatGPT. I am a helpful assistant.

user
Hello! What is your name?

assistant
Hello! My name is ChatGPT. How can I assist you today?

user
What is the weather like today in New York City, New York?

assistant
The current weather in New York City, NY is 20 degrees celsius.



In [13]:
messages.append({"role": "user", "content": "Thank you! How should I dress for that kind of temperature?"})

for message in messages:
    print(f"{message['role']}\n{message['content']}\n")


system
My name is ChatGPT. I am a helpful assistant.

user
Hello! What is your name?

assistant
Hello! My name is ChatGPT. How can I assist you today?

user
What is the weather like today in New York City, New York?

assistant
The current weather in New York City, NY is 20 degrees celsius.

user
Thank you! How should I dress for that kind of temperature?



In [14]:
# Call the model again with the updated messages
response = get_chat_completion(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)

print(response)

{
  "id": "chatcmpl-85JdQRtxpxGq9Fr9fUbU1xZUdE1BJ",
  "object": "chat.completion",
  "created": 1696277808,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "For a temperature of 20 degrees Celsius in New York City, NY, it is recommended to dress in light clothing such as a t-shirt or blouse with shorts, a skirt, or lightweight pants. Additionally, you may want to bring a light sweater or jacket in case the temperature drops in the evening. It's always a good idea to check the weather forecast throughout the day for any unexpected changes."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 192,
    "completion_tokens": 81,
    "total_tokens": 273
  }
}


In [15]:
messages.append(response["choices"][0]["message"])

for message in messages:
    print(f"{message['role']}\n{message['content']}\n")

system
My name is ChatGPT. I am a helpful assistant.

user
Hello! What is your name?

assistant
Hello! My name is ChatGPT. How can I assist you today?

user
What is the weather like today in New York City, New York?

assistant
The current weather in New York City, NY is 20 degrees celsius.

user
Thank you! How should I dress for that kind of temperature?

assistant
For a temperature of 20 degrees Celsius in New York City, NY, it is recommended to dress in light clothing such as a t-shirt or blouse with shorts, a skirt, or lightweight pants. Additionally, you may want to bring a light sweater or jacket in case the temperature drops in the evening. It's always a good idea to check the weather forecast throughout the day for any unexpected changes.

